In [37]:
using ITensors, ITensorMPS
import Pkg
Pkg.activate(@__DIR__)
function Q(n)
  H = OpSum()
  H +=  1.0, "Id", 3*n-3, "Id", 3*n-2 
  H +=  1.0, "Z", 3*n-3, "Id", 3*n-2 
  H +=  -1.0, "Id", 3*n-3, "Z", 3*n-2 
  H +=  -1.0, "Z", 3*n-3, "Z", 3*n-2 
end
let

  #constants
  m=1

  # Create 100 spin-one indices
  N = 90
  sites = siteinds("S=1/2", N)
  states = [isodd(n) ? "Up" : "Dn" for n in 1:N]
  psi0 = MPS(sites,states)

  # Input operator terms which define
  # a Hamiltonian matrix, and convert
  # these terms to an MPO tensor network

  #Mass Term
  H_mass = OpSum()
  for j in 2:(N - 1)
    coeff = m/2* (isodd(div(j,3)) ? 1 : -1)

    H_mass += coeff,"Z", j
  end


  #Electric Field Term
  H_elec = OpSum()
  for j in 2:div(N,3)
    H_elec += Q(j)
  end
  H = H_elec + H_mass

  H = MPO(H_mass + H_elec, sites)


  # Plan to do 5 passes or 'sweeps' of DMRG,
  # setting maximum MPS internal dimensions
  # for each sweep and maximum truncation cutoff
  # used when adapting internal dimensions:
  nsweeps = 5
  maxdim = [10, 20, 100, 100, 200]
  cutoff = 1.0e-10

  # Run the DMRG algorithm, returning energy
  # (dominant eigenvalue) and optimized MPS
  energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
  println("Final energy = $energy")

  # for j in 1:N
  #   println("Site $j: ", dims(psi[j]))
  # end
end


  Activating new project at `c:\Users\Aradh\Desktop\coding\697_fall_2025_Max_Aradh_project`


After sweep 1 energy=75.0  maxlinkdim=1 maxerr=0.00E+00 time=0.049
After sweep 2 energy=75.0  maxlinkdim=1 maxerr=0.00E+00 time=0.043
After sweep 3 energy=75.0  maxlinkdim=1 maxerr=0.00E+00 time=0.033
After sweep 4 energy=75.0  maxlinkdim=1 maxerr=0.00E+00 time=0.039
After sweep 5 energy=75.0  maxlinkdim=1 maxerr=0.00E+00 time=0.039
Final energy = 75.0
